In [58]:
import pathlib
from pydantic import BaseModel, HttpUrl
from datetime import datetime, timedelta
import json

In [15]:
realms_path = pathlib.Path.cwd() / 'data' / 'connected-realms'

class Self(BaseModel):
    href: HttpUrl


class FieldLinks(BaseModel):
    self: Self


class ConnectedRealm(BaseModel):
    href: HttpUrl

class Status(BaseModel):
    type: str
    name: str


class Population(BaseModel):
    type: str
    name: str


class Key(BaseModel):
    href: HttpUrl


class Region(BaseModel):
    key: Key
    name: str
    id: int


class Type(BaseModel):
    type: str
    name: str


class Realm(BaseModel):
    id: int
    region: Region
    connected_realm: ConnectedRealm
    name: str
    category: str
    locale: str
    timezone: str
    type: Type
    is_tournament: bool
    slug: str


class MythicLeaderboards(BaseModel):
    href: HttpUrl


class Auctions(BaseModel):
    href: HttpUrl


class ConnectedRealmResponse(BaseModel):
    field_links: FieldLinks 
    id: int
    has_queue: bool
    status: Status
    population: Population
    realms: list[Realm]
    mythic_leaderboards: MythicLeaderboards
    auctions: Auctions

blizz_connected_realms: list[ConnectedRealmResponse] = []
for realm_file in realms_path.iterdir():
    realm = ConnectedRealmResponse.parse_file(realm_file)
    blizz_connected_realms.append(realm)


In [13]:
class ConnectedRealmRecord(BaseModel):
    id: int
    population_type: str
    mythic_leaderboards_href: HttpUrl
    auction_href: HttpUrl
    self_href: HttpUrl

class RealmRecord(BaseModel):
    id: int
    region_id: int
    connected_realm_id: int
    name: str
    category: str
    locale: str
    timezone: str
    type: str
    is_tournament: bool
    slug: str

class RegionRecord(BaseModel):
    id: int
    name: str
    self_href: HttpUrl

connected_realms: list[ConnectedRealmRecord] = []
realms: list[RealmRecord] = []
regions: list[RegionRecord] = []

for b_cr in blizz_connected_realms:
    cr = ConnectedRealmRecord(
        id=b_cr.id,
        population_type=b_cr.population.type,
        mythic_leaderboards_href=b_cr.mythic_leaderboards.href,
        auction_href=b_cr.auctions.href,
        self_href=b_cr.field_links.self.href
    )
    connected_realms.append(cr)

    for blizz_realm in b_cr.realms:
        realm = RealmRecord(
            id=blizz_realm.id,
            region_id=blizz_realm.region.id,
            connected_realm_id=b_cr.id,
            name=blizz_realm.name,
            category=blizz_realm.category,
            locale=blizz_realm.locale,
            timezone=blizz_realm.timezone,
            type=blizz_realm.type.type,
            is_tournament=blizz_realm.is_tournament,
            slug=blizz_realm.slug
        )
        region = RegionRecord(
            id=blizz_realm.region.id,
            name=blizz_realm.region.name,
            self_href=blizz_realm.region.key.href,
        ) 

        realms.append(realm)
        regions.append(region)

In [48]:
leaderboards_path = pathlib.Path.cwd() / 'data' / 'leaderboards'

class Map(BaseModel):
    name: str
    id: int


class ProfileRealm(BaseModel):
    key: Key
    id: int
    slug: str


class Profile(BaseModel):
    name: str
    id: int
    realm: ProfileRealm


class Faction(BaseModel):
    type: str


class Specialization(BaseModel):
    key: Key
    id: int


class Member(BaseModel):
    profile: Profile
    faction: Faction
    specialization: Specialization


class Color(BaseModel):
    r: int
    g: int
    b: int
    a: float


class MythicRating(BaseModel):
    color: Color
    rating: float


class LeadingGroup(BaseModel):
    ranking: int
    duration: timedelta
    completed_timestamp: datetime
    keystone_level: int
    members: list[Member]
    mythic_rating: MythicRating


class KeystoneAffix(BaseModel):
    key: Key
    name: str
    id: int


class CurrentAffix(BaseModel):
    keystone_affix: KeystoneAffix
    starting_level: int


class LeaderboardResponse(BaseModel):
    field_links: FieldLinks 
    map: Map
    period: int
    period_start_timestamp: datetime
    period_end_timestamp: datetime
    connected_realm: ConnectedRealm
    leading_groups: list[LeadingGroup] | None
    keystone_affixes: list[CurrentAffix]
    map_challenge_mode_id: int
    name: str

class ConnectedRealmLeaderboardMap(BaseModel):
    connected_realm_id: int
    leaderboard: LeaderboardResponse

In [49]:
connected_realm_leaderboards: list[ConnectedRealmLeaderboardMap] = []
for leaderboard_file in leaderboards_path.iterdir():
    leaderboard = LeaderboardResponse.parse_file(leaderboard_file)

    cr_leaderboard = ConnectedRealmLeaderboardMap(
        connected_realm_id=int(leaderboard_file.name.split('-')[0]),
        leaderboard=leaderboard
    )

    connected_realm_leaderboards.append(cr_leaderboard)




In [50]:
class MythicLeaderboardGroupRecord(BaseModel):
    group_id: str
    realm_id: int
    dungeon_id: int
    period_id: int
    dungeon_name: str
    realm_ranking: int
    duration: timedelta
    completed_timestamp: datetime
    keystone_level: int
    mythic_rating: float
    period_start_timestamp: datetime
    period_end_timestamp: datetime
    leaderboard_href: HttpUrl

class MythicLeaderboardGroupMemberRecord(BaseModel):
    group_key_id: str
    character_id: int
    character_name: str
    realm_id: int
    realm_slug: str
    faction: str
    spec_id: int
    spec_href: HttpUrl

In [77]:
leaderboard: list[MythicLeaderboardGroupRecord] = []
members: list[MythicLeaderboardGroupMemberRecord] = []

for l in connected_realm_leaderboards:

    if l.leaderboard.leading_groups is not None:
        for g in l.leaderboard.leading_groups:
            group_id = f'{l.connected_realm_id}-{l.leaderboard.map_challenge_mode_id}-{l.leaderboard.period}-{int(g.completed_timestamp.timestamp())}-{g.duration.seconds}'

            group = MythicLeaderboardGroupRecord(
                group_id=group_id,
                realm_id=l.connected_realm_id,
                dungeon_id=l.leaderboard.map_challenge_mode_id,
                period_id=l.leaderboard.period,
                dungeon_name=l.leaderboard.name,
                realm_ranking=g.ranking,
                duration=g.duration,
                completed_timestamp=g.completed_timestamp,
                keystone_level=g.keystone_level,
                mythic_rating=g.mythic_rating.rating,
                period_start_timestamp=l.leaderboard.period_start_timestamp,
                period_end_timestamp=l.leaderboard.period_end_timestamp,
                leaderboard_href=l.leaderboard.field_links.self.href,
            )

            leaderboard.append(group)

            for m in g.members:
                member = MythicLeaderboardGroupMemberRecord(
                    group_key_id=group_id,
                    character_id=m.profile.id,
                    character_name=m.profile.name,
                    realm_id=m.profile.realm.id,
                    realm_slug=m.profile.realm.slug,
                    faction=m.faction.type,
                    spec_id=m.specialization.id,
                    spec_href=m.specialization.key.href
                )

                members.append(member)

In [78]:
class ConnectedRealmsTable(BaseModel):
    records: list[ConnectedRealmRecord]

class RealmsTable(BaseModel):
    records: list[RealmRecord]

class RegionsTable(BaseModel):
    records: list[RegionRecord]

class LeaderboardTable(BaseModel):
    records: list[MythicLeaderboardGroupRecord]

class MembersTable(BaseModel):
    records: list[MythicLeaderboardGroupMemberRecord]

In [80]:
connected_realms_table = ConnectedRealmsTable(records=connected_realms)
realms_table = RealmsTable(records=realms)
regions_table = RegionsTable(records=regions)
leaderboard_table = LeaderboardTable(records=leaderboard)
member_table = MembersTable(records=members)



In [81]:
with open('./data/tables/connected-realms.json', 'w') as file:
    file.write(connected_realms_table.json(indent=4))

with open('./data/tables/realms.json', 'w') as file:
    file.write(realms_table.json(indent=4))

with open('./data/tables/regions.json', 'w') as file:
    file.write(regions_table.json(indent=4))

with open('./data/tables/leaderboard.json', 'w') as file:
    file.write(leaderboard_table.json(indent=4))

with open('./data/tables/members.json', 'w') as file:
    file.write(member_table.json(indent=4))